In [1]:
from DiffRed import DiffRed as dr

In [2]:
from tensorflow.keras.datasets.fashion_mnist import load_data

2023-04-24 18:39:11.400626: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-24 18:39:11.443477: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-24 18:39:12.012589: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
(x_train,y_train),(x_test,y_test)=load_data()

In [4]:
A=x_train

In [5]:
A.shape

(60000, 28, 28)

In [6]:
A=A.reshape((60000,784))

In [7]:
A.shape

(60000, 784)

In [8]:
A=A[:2000,:] #sample

In [9]:
from DiffRed.utils import data_scaler

In [10]:
A,_=data_scaler(A)

In [11]:
import numpy.linalg as LA
import numpy as np

In [12]:
LA.norm(A[0,:]),np.mean(A[0,:])

(0.9999999999999984, -1.1328806373726086e-18)

In [13]:
A.shape

(2000, 784)

In [14]:
DiffRed=dr(k1=5,k2=15,opt_metric='m1')

In [15]:
embedding=DiffRed.fit_transform(A=A,max_iter=1000)

(784, 5)


In [16]:
embedding.shape

(2000, 20)

In [17]:
DiffRed.metric #value of the metric after monte carlo search   

4.674997431541872e-05

In [18]:
from DiffRed.utils import stress

In [19]:
stress(A,embedding)

0.09673363678630532

In [20]:
from DiffRed.utils import m1

In [21]:
m1(A,embedding)

1.573686867550883e-05